# Convolutional Neural Network for Image Classification

Zewei Chu zeweichu@gmail.com

Reference
- [Stanford CS231n](http://cs231n.github.io/convolutional-networks/)
- [AlexNet](http://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf)
- [VGG](https://arxiv.org/pdf/1409.1556.pdf)
- [ResNet](https://arxiv.org/pdf/1512.03385.pdf)
- [DenseNet](https://arxiv.org/pdf/1608.06993.pdf)



In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
print("PyTorch Version: ",torch.__version__)

PyTorch Version:  1.0.0


Define a model based on ConvNet

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1) # 28 * 28 -> (28+1-5) 24 * 24
        self.conv2 = nn.Conv2d(20, 50, 5, 1) # 20 * 20
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)
        
    def forward(self, x):
        # x: 1 * 28 * 28
        x = F.relu(self.conv1(x)) # 20 * 24 * 24
        x = F.max_pool2d(x,2,2) # 12 * 12
        x = F.relu(self.conv2(x)) # 8 * 8
        x = F.max_pool2d(x,2,2) # 4 *4 
        x = x.view(-1, 4*4*50) # reshape (5 * 2 * 10), view(5, 20) -> (5 * 20)
        x = F.relu(self.fc1(x))
        x= self.fc2(x)
        # return x
        return F.log_softmax(x, dim=1) # log probability
        

In [3]:
mnist_data = datasets.MNIST("./mnist_data", train=True, download=True,
                           transform=transforms.Compose([
                               transforms.ToTensor(),
                           ]))
mnist_data

Processing...
Done!


Dataset MNIST
    Number of datapoints: 60000
    Split: train
    Root Location: ./mnist_data
    Transforms (if any): Compose(
                             ToTensor()
                         )
    Target Transforms (if any): None

In [4]:
data = [d[0].data.cpu().numpy() for d in mnist_data]

In [22]:
np.mean(data)

0.13066062

In [23]:
np.std(data)

0.30810776

In [19]:
mnist_data[223][0].shape

torch.Size([1, 28, 28])

NLL loss

$\ell(x, y) = L = \{l_1,\dots,l_N\}^\top, \quad
        l_n = - w_{y_n} x_{n,y_n}, \quad
        w_{c} = \text{weight}[c] \cdot \mathbb{1}\{c \not= \text{ignore\_index}\}$

In [5]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        pred = model(data) # batch_size * 10
        loss = F.nll_loss(pred, target) 
        
        # SGD
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if idx % 100 == 0:
            print("Train Epoch: {}, iteration: {}, Loss: {}".format(
                epoch, idx, loss.item()))

In [6]:
def test(model, device, test_loader):
    model.eval()
    total_loss = 0.
    correct = 0.
    with torch.no_grad():
        for idx, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)

            output = model(data) # batch_size * 10
            total_loss += F.nll_loss(output, target, reduction="sum").item() 
            pred = output.argmax(dim=1) # batch_size * 1
            correct += pred.eq(target.view_as(pred)).sum().item()
            

    total_loss /= len(test_loader.dataset)
    acc = correct/len(test_loader.dataset) * 100.
    print("Test loss: {}, Accuracy: {}".format(total_loss, acc))

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 32
train_dataloader = torch.utils.data.DataLoader(
    datasets.MNIST("./mnist_data", train=True, download=True,
           transform=transforms.Compose([
               transforms.ToTensor(),
               transforms.Normalize((0.1307,), (0.3081,))
           ])),
    batch_size=batch_size, shuffle=True, 
    num_workers=1, pin_memory=True
)
test_dataloader = torch.utils.data.DataLoader(
    datasets.MNIST("./mnist_data", train=False, download=True,
           transform=transforms.Compose([
               transforms.ToTensor(),
               transforms.Normalize((0.1307,), (0.3081,))
           ])),
    batch_size=batch_size, shuffle=True, 
    num_workers=1, pin_memory=True
)

lr = 0.01
momentum  = 0.5
model = Net().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)

num_epochs = 2
for epoch in range(num_epochs):
    train(model, device, train_dataloader, optimizer, epoch)
    test(model, device, test_dataloader)
    
torch.save(model.state_dict(), "mnist_cnn.pt")

Train Epoch: 0, iteration: 0, Loss: 2.3180577754974365
Train Epoch: 0, iteration: 100, Loss: 0.5994296669960022
Train Epoch: 0, iteration: 200, Loss: 0.24825681746006012
Train Epoch: 0, iteration: 300, Loss: 0.3089035749435425
Train Epoch: 0, iteration: 400, Loss: 0.4593719244003296
Train Epoch: 0, iteration: 500, Loss: 0.2870183289051056
Train Epoch: 0, iteration: 600, Loss: 0.1864209622144699
Train Epoch: 0, iteration: 700, Loss: 0.1103847548365593
Train Epoch: 0, iteration: 800, Loss: 0.07073946297168732
Train Epoch: 0, iteration: 900, Loss: 0.34052252769470215
Train Epoch: 0, iteration: 1000, Loss: 0.21388737857341766
Train Epoch: 0, iteration: 1100, Loss: 0.10509373992681503
Train Epoch: 0, iteration: 1200, Loss: 0.15032045543193817
Train Epoch: 0, iteration: 1300, Loss: 0.03676900267601013
Train Epoch: 0, iteration: 1400, Loss: 0.14294494688510895
Train Epoch: 0, iteration: 1500, Loss: 0.2562383711338043
Train Epoch: 0, iteration: 1600, Loss: 0.005333080887794495
Train Epoch: 0, 

### More reading


- [Torch Vision Models](https://github.com/pytorch/vision/tree/master/torchvision/models)